<a href="https://colab.research.google.com/github/maverick98/Coursera/blob/master/mfml1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

In [3]:
#Create matrix
def create_random_matrix(m,n,generate_float=False):
    if generate_float == True:
        A = np.random.ranf(size=(m, n))
    else:
        A = np.random.randint(low=0,high=100,size=(m, n))
        A=A.astype(float)
    return A

def augment_matrix(A,b):
    b=b.reshape(b.shape[0],1)
    return np.append(A,b,axis=1)


In [4]:
#Elementary row operations
def const_multiple(A,i,c):
    A[i]*=c
def exchange_rows(A,i,j):
    A[[i,j]] = A[[j,i]]
def add_const_row_multiple(A,i,j,c):
    A[i]+=A[j]*c


In [25]:
def find_pivot_row(A, curr_pivot_row, col):
    rows = A.shape[0]
    for row in range(curr_pivot_row, rows):
        if A[row, col] != 0.0:
            return row
    return None

def convert_pivot_value_to_one(A,pivot_row,col):
    pivot_row_value=A[pivot_row,col]
    A[pivot_row] /=pivot_row_value


def convert_pivot_row_column_values_to_zero_above(A,pivot_row,col):
    rows = A.shape[0]
    for row in range(0,pivot_row):
        const_multiplier=A[row][col]/A[pivot_row][col]
        #print('subtracting {}*A[{}] from A[{}]'.format(const_multiplier, pivot_row,row))
        A[row] -=const_multiplier*A[pivot_row]

def convert_pivot_row_column_values_to_zero_below(A,pivot_row,col):
    rows = A.shape[0]
    for row in range(pivot_row+1, rows):
        const_multiplier=A[row][col]/A[pivot_row][col]

        #print('subtracting {}*A[{}] from A[{}]'.format(const_multiplier, pivot_row,row))
        A[row] -=const_multiplier*A[pivot_row]

def calculate_row_echelon_form(A,rref=False):


    pivot_row_cols=[]
    pivot_cols=[]
    non_pivot_cols=[]
    rows=A.shape[0]
    cols=A.shape[1]

    def calculate_pivot_and_non_pivot_cols():
        all_cols=np.array(range(cols-1))
        pivot_cols_list=[]
        for pivot_row_col in pivot_row_cols:
            pivot_cols_list.append(pivot_row_col[1])
        pivot_cols=np.array(pivot_cols_list)
        non_pivot_cols=np.setdiff1d(all_cols,pivot_cols)
        return pivot_cols,non_pivot_cols


    curr_pivot_row=0
    for col in range(cols):
        nonzero_row = find_pivot_row(A, curr_pivot_row, col)
        if nonzero_row is not None:
            exchange_rows(A,curr_pivot_row,nonzero_row)
            #print('pivot row={} column={} '.format(curr_pivot_row,col))
            pivot_row_cols.append((curr_pivot_row,col))
            if rref == True:
                convert_pivot_value_to_one(A,curr_pivot_row,col)
            convert_pivot_row_column_values_to_zero_below(A,curr_pivot_row,col)
            if rref == True:
                convert_pivot_row_column_values_to_zero_above(A,curr_pivot_row,col)
            curr_pivot_row+=1
    pivot_cols,non_pivot_cols=calculate_pivot_and_non_pivot_cols()
    return pivot_row_cols,pivot_cols,non_pivot_cols

def calculate_generic_solution(A,b):
    if A.shape[0] != b.shape[0]:
        print("Illegal values for A and b")
        return
    print('Matrix is')
    print(A)
    print('b is')
    print(b)

    A_aug=augment_matrix(A,b)
    print('Augmented matrix is')
    print(A_aug)
    #print('REF matrix is '.format(A_aug))
    pivot_row_cols,pivot_cols , non_pivot_cols = calculate_row_echelon_form(A_aug,rref=True)
    print('RREF matrix is')
    print(A_aug)
    #print('pivot_row_cols is {}'.format(pivot_row_cols))
    print('pivot_cols is {}'.format(pivot_cols))
    print('non_pivot_cols is {}'.format(non_pivot_cols))
    x_particular=np.zeros(A.shape[1],dtype=float)
    x_nullspace_solutions=[]
    last_col=A_aug.shape[1]-1
    for pivot_row_col in pivot_row_cols:
        x_particular[pivot_row_col[1]]=A_aug[ pivot_row_col[0],last_col]

    const_multiplier_prefix='c'
    const_multiplier_idx=1
    x_nullspace_solutions_list=[]
    for non_pivot_col in non_pivot_cols:
        x_nullspace_solution=np.zeros(A.shape[1],dtype=float)
        x_nullspace_solution[non_pivot_col]=-1
        idx=0
        for pivot_col in pivot_cols:
            if pivot_col < non_pivot_col:
               x_nullspace_solution[pivot_col]=A_aug[idx,non_pivot_col]
               idx+=1
        x_nullspace_solution_str=' + {}*{}'.format(const_multiplier_prefix+str(const_multiplier_idx),x_nullspace_solution)
        const_multiplier_idx+=1
        x_nullspace_solutions.append(x_nullspace_solution)
        x_nullspace_solutions_list.append(x_nullspace_solution_str)
    x_nullspace_solutions_str=' '.join(x_nullspace_solutions_list)
    x_generic_solution_str='Generic solution would be {} {}'.format(x_particular,x_nullspace_solutions_str)

    return  x_particular,x_nullspace_solutions,x_generic_solution_str



In [6]:
def create_matrix():
    A = np.array([[ 1, 0, 8 ,-4 ], [0 ,1, 2, 12]])
    A=A.astype(float)
    return A

In [13]:
def answer(A,b):
    #print(A)
    x_particular,x_nullspace_solutions,x_generic_solution_str=calculate_generic_solution(A,b)
    print('Particular solution is {}'.format(x_particular))
    print('NullSpace solution(s) are {}'.format(x_nullspace_solutions))
    print(x_generic_solution_str)



In [26]:
A=create_matrix()
b=np.array([42,8]).astype(float)
answer(A,b)


Matrix is
[[ 1.  0.  8. -4.]
 [ 0.  1.  2. 12.]]
b is
[42.  8.]
Augmented matrix is
[[ 1.  0.  8. -4. 42.]
 [ 0.  1.  2. 12.  8.]]
RREF matrix is
[[ 1.  0.  8. -4. 42.]
 [ 0.  1.  2. 12.  8.]]
pivot_cols is [0 1]
non_pivot_cols is [2 3]
Particular solution is [42.  8.  0.  0.]
NullSpace solution(s) are [array([ 8.,  2., -1.,  0.]), array([-4., 12.,  0., -1.])]
Generic solution would be [42.  8.  0.  0.]  + c1*[ 8.  2. -1.  0.]  + c2*[-4. 12.  0. -1.]


In [29]:
A = np.array([[ 2, 3 ], [1 , -1]])
A=A.astype(float)
b=np.array([22,1]).astype(float)
answer(A,b)

Matrix is
[[ 2.  3.]
 [ 1. -1.]]
b is
[22.  1.]
Augmented matrix is
[[ 2.  3. 22.]
 [ 1. -1.  1.]]
RREF matrix is
[[ 1.  0.  5.]
 [-0.  1.  4.]]
pivot_cols is [0 1]
non_pivot_cols is []
Particular solution is [5. 4.]
NullSpace solution(s) are []
Generic solution would be [5. 4.] 


In [30]:
A = np.array([[ 2, 3 ], [4 , 6]])
A=A.astype(float)
b=np.array([22,44]).astype(float)
answer(A,b)

Matrix is
[[2. 3.]
 [4. 6.]]
b is
[22. 44.]
Augmented matrix is
[[ 2.  3. 22.]
 [ 4.  6. 44.]]
RREF matrix is
[[ 1.   1.5 11. ]
 [ 0.   0.   0. ]]
pivot_cols is [0]
non_pivot_cols is [1]
Particular solution is [11.  0.]
NullSpace solution(s) are [array([ 1.5, -1. ])]
Generic solution would be [11.  0.]  + c1*[ 1.5 -1. ]


In [27]:
A=create_random_matrix(5,7)
b=np.array([112,13,131,78,89]).astype(float)
answer(A,b)

Matrix is
[[28. 16. 31. 39. 41. 45.  3.]
 [23. 97. 83. 91. 86. 69. 68.]
 [86. 24.  8. 58. 39.  3. 36.]
 [ 8. 85. 67. 68. 76. 50. 69.]
 [96. 14. 10. 17.  0. 92. 49.]]
b is
[112.  13. 131.  78.  89.]
Augmented matrix is
[[ 28.  16.  31.  39.  41.  45.   3. 112.]
 [ 23.  97.  83.  91.  86.  69.  68.  13.]
 [ 86.  24.   8.  58.  39.   3.  36. 131.]
 [  8.  85.  67.  68.  76.  50.  69.  78.]
 [ 96.  14.  10.  17.   0.  92.  49.  89.]]
RREF matrix is
[[ 1.          0.          0.          0.          0.          0.99632775
   0.52423828  2.41618058]
 [ 0.          1.          0.          0.          0.         -0.43537284
   1.56490733 -0.70443366]
 [ 0.          0.          1.          0.          0.          2.90496855
  -0.49886293 -1.50527148]
 [-0.         -0.         -0.          1.          0.         -1.56481941
  -1.07334988 -6.9434441 ]
 [-0.         -0.         -0.         -0.          1.         -0.12090926
   0.5026392   9.09940532]]
pivot_cols is [0 1 2 3 4]
non_pivot_cols is [